In [1]:
import os
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession
import json
import re
import logging
from pyspark.sql.window import Window
import operator

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1619342430964_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.appName("Deloitte Challenge Test").config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## JSON FILES PATHS

In [3]:
stations_data = "s3a://deloitte-ciprian-test/data/source/stations.json"
trips_data = "s3a://deloitte-ciprian-test/data/source/trips.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read Data From Files

In [7]:
df_stations = spark.read.option("multiline", "true").json(stations_data)
df_stations.show(truncate=False, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------
 internal_bus_station_id | [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 5]                                                                                                   
 public_bus_station      | [BAutogara, BVAutogara, SBAutogara, CJAutogara, MMAutogara, ISAutogara, CTAutogara, TMAutogara, BCAutogara, MSAutogara, ISAutogara]

### Read data from Trips File using Schema

In [9]:
schema = T.StructType([
    T.StructField("ORIGIN", T.ArrayType(T.StringType(), True), True),
    T.StructField("DESTINATION", T.ArrayType(T.StringType(), True), True),
    T.StructField("INTERNAL_BUS_STATION_IDS", T.ArrayType(T.ArrayType(T.IntegerType())), True),
    T.StructField("TRIPTIMES", T.ArrayType(T.ArrayType(T.StringType(), True), True), True),
])

df_trips = spark.read.schema(schema).option("multiline", "true")\
.option("mode", "PERMISSIVE")\
.option("columnNameOfCorruptRecord", "_corrupt_record").json(trips_data)
df_trips.show(truncate=False, vertical=True)
df_trips.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 ORIGIN                   | [B, BV, TM, CJ]                                                                                                                                                                                                                                                      
 DESTINATION              | [MM, IS, CT, BC]                                                                                                                                                                                                                                                     
 INTERNAL_BUS_STATION_IDS | [[0, 2, 4], [1, 8, 5], [7, 2, 6], [3, 9, 8]]                                                          

### EXPLODE the arrays from station File coresponding to id and name station columns

In [10]:
df_internal_bus_station_id_exploded = df_stations.select(F.posexplode(F.col('internal_bus_station_id')))
df_bus_station_exploded = df_stations.select(F.posexplode(F.col('public_bus_station')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
inner_stations_join = df_internal_bus_station_id_exploded.select(F.col('col').alias('internal_bus_station_id'), F.col('pos')).join(df_bus_station_exploded.select(F.col('col').alias('public_bus_station'), F.col('pos')), df_internal_bus_station_id_exploded.pos == df_bus_station_exploded.pos, how='inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## STATIONS TABLE

In [12]:
inner_stations_join = inner_stations_join.select("internal_bus_station_id", "public_bus_station")
inner_stations_distinct_join = inner_stations_join.drop_duplicates(subset=['internal_bus_station_id'])
inner_stations_distinct_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### EXPLODE the External Array from Trips File coresponding to INTERNAL_BUS_STATION_IDS column

In [34]:
df_trips_internal_bus_station_ids_exploded = df_trips.select(F.posexplode(F.col('INTERNAL_BUS_STATION_IDS')))
df_trips_internal_bus_station_ids_exploded = df_trips_internal_bus_station_ids_exploded.select(F.col("pos").alias("tripkey"), F.col("col").alias("bus_stations"))
df_trips_internal_bus_station_ids_exploded.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
|tripkey|bus_stations|
+-------+------------+
|      0|   [0, 2, 4]|
|      1|   [1, 8, 5]|
|      2|   [7, 2, 6]|
|      3|   [3, 9, 8]|
+-------+------------+

### EXPLODE the Arrays from Trips File coresponding to ORIGIN, DESTINATION and TRIPTIMES columns

In [28]:
df_trips_origin = df_trips.select(F.posexplode(F.col('ORIGIN')))
df_trips_dest = df_trips.select(F.posexplode(F.col('DESTINATION')))
df_trips_triptimes = df_trips.select(F.posexplode(F.col('TRIPTIMES')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## JOIN TABLES to create the INITIAL Table

In [29]:
df_trips_origin.createOrReplaceTempView("origin_table")
df_trips_dest.createOrReplaceTempView("dest_table")
df_trips_triptimes.createOrReplaceTempView("triptimes_table")
df_trips_internal_bus_station_ids_exploded.createOrReplaceTempView("bus_stations_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# INITIAL TABLE

In [30]:
initial_table = spark.sql("""
        SELECT 
        ROW_NUMBER() OVER(ORDER BY a.tripkey ASC) AS row_num,
        b.col as origin, 
        c.col as destination, 
        a.bus_stations as bus_stations,
        d.col as triptimes
        FROM bus_stations_table as a
        JOIN origin_table as b ON a.tripkey = b.pos
        JOIN dest_table as c ON b.pos=c.pos
        JOIN triptimes_table as d ON a.tripkey=d.pos
        ORDER BY a.tripkey
        """)
initial_table.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+-----------+------------+---------------------------------------------------------------+
|row_num|origin|destination|bus_stations|triptimes                                                      |
+-------+------+-----------+------------+---------------------------------------------------------------+
|1      |B     |MM         |[0, 2, 4]   |[2020-03-01 10:10:00, 2020-03-01 12:20:10, 2020-03-01 15:10:00]|
|2      |BV    |IS         |[1, 8, 5]   |[2020-03-01 10:10:00, 2020-03-01 12:20:10, 2020-03-01 15:10:00]|
|3      |TM    |CT         |[7, 2, 6]   |[2020-04-01 10:10:00, 2020-04-01 12:20:10, 2020-04-01 15:10:00]|
|4      |CJ    |BC         |[3, 9, 8]   |[2020-05-01 10:10:00, 2020-05-01 12:20:10, 2020-05-01 15:10:00]|
+-------+------+-----------+------------+---------------------------------------------------------------+

### EXPLODE the Internal Array from Trips File coresponding to INTERNAL_BUS_STATION_IDS column

In [35]:
df_trips_internal_bus_station_ids_exploded = df_trips_internal_bus_station_ids_exploded.select(F.col('tripkey'), F.posexplode(F.col('bus_stations')))
df_trips_internal_bus_station_ids_exploded.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---+---+
|tripkey|pos|col|
+-------+---+---+
|      0|  0|  0|
|      0|  1|  2|
|      0|  2|  4|
|      1|  0|  1|
|      1|  1|  8|
|      1|  2|  5|
|      2|  0|  7|
|      2|  1|  2|
|      2|  2|  6|
|      3|  0|  3|
|      3|  1|  9|
|      3|  2|  8|
+-------+---+---+

### Merge the bus stations Id columns with names grouped and ordered for each trip

In [37]:
df_stations_ids_names = df_trips_internal_bus_station_ids_exploded.join(inner_stations_distinct_join, inner_stations_distinct_join.internal_bus_station_id==df_trips_internal_bus_station_ids_exploded.col, how="inner")
df_stations_ids_names.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---+---+-----------------------+------------------+
|tripkey|pos|col|internal_bus_station_id|public_bus_station|
+-------+---+---+-----------------------+------------------+
|      0|  0|  0|                      0|         BAutogara|
|      2|  0|  7|                      7|        TMAutogara|
|      2|  2|  6|                      6|        CTAutogara|
|      3|  1|  9|                      9|        MSAutogara|
|      1|  2|  5|                      5|        ISAutogara|
|      1|  0|  1|                      1|        BVAutogara|
|      3|  0|  3|                      3|        CJAutogara|
|      3|  2|  8|                      8|        BCAutogara|
|      1|  1|  8|                      8|        BCAutogara|
|      2|  1|  2|                      2|        SBAutogara|
|      0|  1|  2|                      2|        SBAutogara|
|      0|  2|  4|                      4|        MMAutogara|
+-------+---+---+-----------------------+------------------+

### Order data in each array after apply collect list

In [40]:
def sorter(l):
  res = sorted(l, key=operator.itemgetter(0))
  return [item[1] for item in res]

sort_udf = F.udf(sorter, T.ArrayType(T.StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Bus Stations Names for Final Table

In [41]:
df_stations_ids_names_agg = df_stations_ids_names.groupBy('tripkey').agg(F.collect_list(F.struct("pos", "public_bus_station")).alias("bus_unordered"))
df_stations_ids_names_agg = df_stations_ids_names_agg.withColumn("bus_stations", sort_udf(F.col("bus_unordered"))).drop("bus_unordered")
df_stations_ids_names_agg.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------------------------+
|tripkey|bus_stations                        |
+-------+------------------------------------+
|1      |[BVAutogara, BCAutogara, ISAutogara]|
|3      |[CJAutogara, MSAutogara, BCAutogara]|
|2      |[TMAutogara, SBAutogara, CTAutogara]|
|0      |[BAutogara, SBAutogara, MMAutogara] |
+-------+------------------------------------+

### UDF function to calculate the duration of each trip

In [42]:
import datetime
def calculate_journey_duration(column):
    last_date = datetime.datetime.strptime(column[len(column) - 1], '%Y-%m-%d %H:%M:%S')
    first_date = datetime.datetime.strptime(column[0], '%Y-%m-%d %H:%M:%S')
    dif_result =  last_date - first_date
    journey_time_minutes = int(dif_result.seconds / 60)
    return journey_time_minutes  
    
calculate_journey_udf = F.udf(calculate_journey_duration, T.IntegerType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
df_trips_triptimes_agg = df_trips_triptimes.withColumn("min", calculate_journey_udf(F.col('col')).cast("integer"))
df_trips_triptimes_agg.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------------------------------------------------------------+---+
|pos|col                                                            |min|
+---+---------------------------------------------------------------+---+
|0  |[2020-03-01 10:10:00, 2020-03-01 12:20:10, 2020-03-01 15:10:00]|300|
|1  |[2020-03-01 10:10:00, 2020-03-01 12:20:10, 2020-03-01 15:10:00]|300|
|2  |[2020-04-01 10:10:00, 2020-04-01 12:20:10, 2020-04-01 15:10:00]|300|
|3  |[2020-05-01 10:10:00, 2020-05-01 12:20:10, 2020-05-01 15:10:00]|300|
+---+---------------------------------------------------------------+---+

## JOIN TABLES to create the FINAL Table

In [47]:
df_stations_ids_names_agg.createOrReplaceTempView("final_bus_stations_table")
df_trips_triptimes_agg.createOrReplaceTempView("final_duration_table")
final_table = spark.sql("""
        SELECT 
        ROW_NUMBER() OVER(ORDER BY a.tripkey ASC) AS row_num,
        b.col as origin, 
        c.col as destination, 
        a.bus_stations as bus_stations,
        d.min as duration
        FROM final_bus_stations_table as a
        JOIN origin_table as b ON a.tripkey = b.pos
        JOIN dest_table as c ON b.pos=c.pos
        JOIN final_duration_table as d ON a.tripkey=d.pos
        ORDER BY a.tripkey
        """)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# FINAL TABLE

In [48]:
final_table.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+-----------+------------------------------------+--------+
|row_num|origin|destination|bus_stations                        |duration|
+-------+------+-----------+------------------------------------+--------+
|1      |B     |MM         |[BAutogara, SBAutogara, MMAutogara] |300     |
|2      |BV    |IS         |[BVAutogara, BCAutogara, ISAutogara]|300     |
|3      |TM    |CT         |[TMAutogara, SBAutogara, CTAutogara]|300     |
|4      |CJ    |BC         |[CJAutogara, MSAutogara, BCAutogara]|300     |
+-------+------+-----------+------------------------------------+--------+